<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# A ver si nos hacemos ricos (LSTM edition)<a id="top"></a>

<i>Última actualización: 2024-03-07</small></i></div>

***

## Introducción

El intento anterior de predecir el mercado de valores no salió como esperábamos. Vamos a intentarlo de nuevo, esta vez con unas redes LSTM completamente nuevas, y utilizando únicamente el valor bursátil anterior.

## Objetivos

Crearemos un modelo de regresión para predecir un valor de bolsa dado el valor anterior. Al final habremos aprendido a:

- Predecir la tendencia de una serie temporal utilizando LSTM.
- Pensar dos veces antes de invertir en bolsa en base a las recomendaciones realizadas por los modelos que hemos programado.

## Bibliotecas y configuración

A continuación importaremos las librerías que se utilizarán a lo largo del cuaderno.

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Carga y preparación de datos

Recargaremos los datos del historial de Google como hicimos en el anterior intento de predicción bursátil.

In [ ]:
CODE = 'GOOG'
BASE_URL = f'https://query1.finance.yahoo.com/v7/finance/download/{CODE}'
# Interval
today = datetime.datetime.now()
five_years_ago = today - datetime.timedelta(days=365*5)
# Timestamps (as integer) for the interval
period2 = int(today.timestamp())
period1 = int(five_years_ago.timestamp())
# Now get the dataframe
df = pd.read_csv(
    f'{BASE_URL}?period1={period1}&period2={period2}&interval=1d',
    index_col='Date',
    parse_dates=['Date'],
    dtype=np.float32
)
df.head()

Vamos a omitir el resto de valores y a centrarnos únicamente en la evolución temporal del valor de cierre de la acción. Eso sí, en lugar de utilizar el valor absoluto, lo que utilizaremos será la variación del valor respecto al día anterior. No vamos a hacer ninguna transformación más allá de normalizar y poco más, por lo que podemos convertirlo a un array de numpy sin problemas.

In [ ]:
dataset = df[['Close']].diff(axis=0).values[1:]
dataset

Ahora crearemos un `MinMaxScaler` para hacer la normalización y desnormalización de nuestros datos. Aprovechamos y los dejamos ya normalizados.

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
dataset = scaler.fit_transform(dataset)
dataset

Ya tenemos nuestro conjunto de datos casi listo para trabajar. Hemos transformado todo el conjunto de datos para hacer el ejercicio más corto, pero recuerde que en la normalización de los datos no deben entrar los datos de prueba El siguiente paso es especificar la longitud de la secuencia, que es lo mismo que el número de pasos temporales o el número de observaciones previas a considerar para hacer una predicción.

Utilizaremos un tamaño de 20, lo que significa que para predecir el valor de un día se utilizarán los 20 anteriores (aproximadamente un mes). Para ello crearemos los conjuntos `X_train` y `Y_train`, dos arrays NumPy que contendrán las secuencias y el siguiente valor de esa secuencia respectivamente.

In [ ]:
SEQUENCE_LEN = 10

x_train = []
y_train = []
for i in range(len(dataset) - SEQUENCE_LEN - 9):
    x_train.append(dataset[i:i + SEQUENCE_LEN, 0])
    y_train.append(dataset[i + SEQUENCE_LEN + 9, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)

for i in range(10):
    print(f'... {x_train[i][-3:]} -> {y_train[i]}')
print('-'*72)
print(f'x_train shape: {x_train.shape}, Y_train shape: {y_train.shape}')

Recordemos que los datos de entrada de una red recurrente deben tener una dimensión específica, por lo que ahora los transformaremos para adaptarlos a este requisito.

In [ ]:
x_train = np.reshape(x_train, (*x_train.shape[:2], 1))

print(f'X_train shape: {x_train.shape}')

Por último, ahora que tenemos los datos listos, extraigamos los últimos valores (por ejemplo, 100) para que sirvan como conjunto de prueba

In [ ]:
TEST_SIZE = 250

x_train, x_test = x_train[:-TEST_SIZE], x_train[-TEST_SIZE:]
y_train, y_test = y_train[:-TEST_SIZE], y_train[-TEST_SIZE:]

print(f'X_train shape: {x_train.shape}, Y_train shape: {y_train.shape}')
print(f'X_test shape: {x_test.shape}, Y_test shape: {y_test.shape}')

## Implementación y entrenamiento del modelo

Pasamos a construir nuestra red neuronal recurrente. Crearemos una estructura apilada de 3 nodos `LSTM` de 40 nodos cada uno, con una capa `Dropout` después de cada capa. A continuación, utilizaremos como optimizador _Adam_ y como medida de pérdida el error cuadrático medio.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['root_mean_squared_error'])
model.summary()

Ahora, entrenaremos el modelo durante... digamos $100$ _epochs_, con los datos de entrenamiento que hemos preparado.

In [ ]:
history = model.fit(x_train, y_train, epochs=100)

Echemos un vistazo a la evolución del entrenamiento.

In [ ]:
pd.DataFrame(history.history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

## Evaluación del modelo

Una vez hecho todo, éste es el paso fácil. Tenemos que pasar nuestras fechas de prueba para ver cómo se comporta nuestro modelo con valores que nunca ha visto. Los compararemos con los valores reales para ver lo bien que lo hace.

In [ ]:
predicted = model.predict(x_test)
#predicted = scaler.inverse_transform(predicted)
#real = scaler.inverse_transform(y_test.reshape((-1, 1)))
real = y_test.reshape((-1, 1))

plt.plot(real, label = 'Real')
plt.plot(predicted, label = 'Predicted')
plt.title('GOOG Stock Price Prediction')
plt.xlabel('Day')
plt.ylabel('GOOG Stock Price')
plt.legend()
plt.show()

Al menos no es un comportamiento como el anterior. Pero sigue siendo inútil. Aparentemente no vamos a hacernos ricos, al menos no así.

## Conclusiones

En este _notebook_ hemos aprendido que es muy fácil cambiar entre tipos de unidades recurrentes. La API de Keras nos permite sustituir uno y otro prácticamente sin esfuerzo.

También hemos visto que no todos los problemas de series temporales son resolubles, al menos no con los datos que tenemos a mano en un principio. Por supuesto que el valor bursátil de una acción es una serie temporal, pero hay muchas variables relacionadas con los valores, muchas de ellas emocionales de los propios propietarios de las acciones, por lo que es muy difícil identificarlas, obtenerlas y cuantificarlas. Aun así, si lo pruebas y te sale muy bien, no te olvides de nosotros.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>